!pip install grpcio==1.42.0 tensorflow-serving-api==2.7.0

In [39]:
import grpc
import tensorflow as tf

from tensorflow_serving.apis import predict_pb2
from tensorflow_serving.apis import prediction_service_pb2_grpc

from io import BytesIO
from urllib import request
import numpy as np
from PIL import Image

In [40]:
host = 'localhost:8500'

channel = grpc.insecure_channel(host , options=(('grpc.enable_http_proxy', 0),))

stub = prediction_service_pb2_grpc.PredictionServiceStub(channel)


In [41]:
stub

In [42]:
url = "http://bit.ly/mlbookcamp-pants"

In [43]:
classes = ['dress',
        'hat',
        'longsleeve',
        'outwear',
        'pants',
        'shirt',
        'shoes',
        'shorts',
        'skirt',
        't-shirt']

def download_image(url):
    with request.urlopen(url) as resp:
        buffer = resp.read()
    stream = BytesIO(buffer)
    img = Image.open(stream)
    return img

def preprocess_input(url):
    x = download_image(url)
    x = x.resize((299, 299), Image.NEAREST)
    arr = np.array(x, dtype="float32")
    arr = np.array([arr])
    arr /= 127.5
    arr -= 1.
    return arr

def decode_result(pred):
    result = {c: float(p) for c, p in zip(classes, pred)}
    return result


In [44]:
X = preprocess_input(url)

C:\Users\soumy\AppData\Local\Temp\ipykernel_6156\2201985149.py:21: DeprecationWarning: NEAREST is deprecated and will be removed in Pillow 10 (2023-07-01). Use Resampling.NEAREST or Dither.NONE instead.
  x = x.resize((299, 299), Image.NEAREST)


In [45]:
def numpy_to_protobuf(data):
    return tf.make_tensor_proto(data, shape=data.shape)

In [48]:
pb_request = predict_pb2.PredictRequest()

#   from model-description.txt
pb_request.model_spec.name = 'clothing-model'
pb_request.model_spec.name = 'serving_default'
pb_request.inputs['input_28'].CopyFrom(numpy_to_protobuf(X))

In [49]:
pb_response = stub.Predict(pb_request, timeout=20.0)

_InactiveRpcError: <_InactiveRpcError of RPC that terminated with:
	status = StatusCode.NOT_FOUND
	details = "Servable not found for request: Latest(serving_default)"
	debug_error_string = "{"created":"@1672596241.003000000","description":"Error received from peer \u00e0x\u00d5\u0001\u00a7\u0001","file":"src/core/lib/surface/call.cc","file_line":1064,"grpc_message":"Servable not found for request: Latest(serving_default)","grpc_status":5}"
>